In [6]:
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
import tqdm
from multiprocess import Pool
from pandas import DataFrame as df
from os.path import join
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import ipycytoscape
import json

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
dirname = "../extavy/beem"
frames_list = [filename for filename in os.listdir(dirname) if filename.endswith(".aig")]
max_pool = 12

        
def process_aig(f):
    os.system("cd ../extavy/build/avy/src && ./avypdr -L 1 -v 1 " + os.path.join("../../../beem/", f) + " > " + os.path.join("../../../beem", f.split(".")[0] + "res.txt"))
    return f

with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm.tqdm(
            p.imap(process_aig,
                   frames_list),
            total=len(frames_list)
        )
    )
    print("\n".join(pool_outputs))

In [11]:
with open("../pass/pass/pass1.json") as fi:
    json_file = json.load(fi)

In [12]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(json_file)

In [13]:
cytoscapeobj.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
cytoscapeobj.set_style([{
                        'selector': 'node',
                        'css': {
                            'background-color': '#11479e'
                            }
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'width': 4,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'curve-style': 'bezier'
                            }
                        }])

In [14]:
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLengthVal': 10}, cytoscape_style=[{…